In [ ]:
#---#| default_exp peptide.precursor

# Precursor Functionalities

In [ ]:
from alphabase.peptide.precursor import *

### Testing

In [ ]:
repeat = 2
peptides = ['AGHCEWQMKAADER']*repeat
mods = ['Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M']*repeat
sites = ['0;4;8']*repeat
peptides += ['AGHCEWQMK']*repeat
mods += ['']*repeat
sites += ['']*repeat

precursor_df = pd.DataFrame({
    'sequence': peptides,
    'mods': mods,
    'mod_sites': sites
})
precursor_df['nAA'] = precursor_df['sequence'].str.len()
precursor_df['charge'] = 2

In [ ]:
#| hide
calc_precursor_isotope(precursor_df)
assert np.allclose(
    precursor_df.precursor_mz.values, 
    [873.86977, 873.86977,545.233862, 545.233862]
)
assert abs(
    precursor_df.loc[1,'isotope_m1_mz']-precursor_df.loc[1,'precursor_mz']-MASS_ISOTOPE/2
)<=1e-5
assert abs(
    precursor_df.loc[1,'isotope_apex_mz']-precursor_df.loc[1,'precursor_mz']
    -MASS_ISOTOPE/precursor_df.loc[1,'isotope_apex_offset'] 
    if precursor_df.loc[1,'isotope_apex_offset'] > 0 else 0
)<=1e-5
precursor_df

,sequence,mods,mod_sites,nAA,charge,precursor_mz,isotope_m1_intensity,isotope_apex_intensity,isotope_apex_offset,isotope_right_most_intensity,isotope_right_most_offset,isotope_m1_mz,isotope_apex_mz,isotope_right_most_mz
0,AGHCEWQMKAADER,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,14,2,873.869771,0.888952,1.0,0,0.235889,3,874.371421,873.869771,875.374721
1,AGHCEWQMKAADER,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,14,2,873.869771,0.888952,1.0,0,0.235889,3,874.371421,873.869771,875.374721
2,AGHCEWQMK,,,9,2,545.233862,0.576623,1.0,0,0.277542,2,545.735512,545.233862,546.237162
3,AGHCEWQMK,,,9,2,545.233862,0.576623,1.0,0,0.277542,2,545.735512,545.233862,546.237162


In [ ]:
#| hide
def detect_duplicated_items(
    precursor_df:pd.DataFrame, 
    item_column:str='mod_seq_charge_hash'
)->pd.Index:
    return precursor_df.loc[
        precursor_df[item_column].duplicated()
    ].index

hash_precursor_df(precursor_df)
assert (detect_duplicated_items(precursor_df, 'mod_seq_hash').values==(1,3)).all()
assert (detect_duplicated_items(precursor_df, 'mod_seq_charge_hash').values==(1,3)).all()
assert precursor_df['mod_seq_hash'].dtype == np.int64
assert precursor_df['mod_seq_charge_hash'].dtype == np.int64
assert get_mod_seq_hash("AGHCEWQMK",'','') == precursor_df.mod_seq_hash.values[-1]
assert get_mod_seq_hash("AGHCEWQMKAADER",'Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M','0;4;8') == precursor_df.mod_seq_hash.values[0]
assert get_mod_seq_charge_hash("AGHCEWQMK",'','',2) == precursor_df.mod_seq_charge_hash.values[-1]
assert get_mod_seq_charge_hash("AGHCEWQMKAADER",'Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M','0;4;8',2) == precursor_df.mod_seq_charge_hash.values[0]